In [13]:
# Importo librerias necesarias
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from google.cloud import bigquery

#from pyspark.sql import SparkSession

In [6]:
url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

In [7]:
response = requests.get(url)

In [8]:
soup = BeautifulSoup(response.content, "html.parser") # Contenido de la pagina
parquet_files = [] # Creo una lista para alojar todos los links que sean de los archivos parquet

In [10]:
# Hago una filtración de los archivos que pertenecen solamente a los años que nos importan
selected_years = ['2017', '2018', '2019']

for link in soup.find_all('a',href=True): # Busco todos los elementos que sean links
  if link['href'].endswith('.parquet'): # Si el elemento termina en .parquet, añado el elemento a la lista parquet_files
    if any(year in link['href'] for year in selected_years):
      parquet_files.append(link['href'])

In [11]:
yellow_taxis = []
green_taxis = []
grey_taxis = []
black_taxis = []

for link in parquet_files:
  if 'yellow_tripdata' in link: # Todos los links que contengan 'yellow_tripdata' en su texto
    yellow_taxis.append(link)
  elif 'green_tripdata' in link: # Todos los links que contengan 'green_tripdata' en su texto
    green_taxis.append(link)
  elif 'fhv_tripdata' in link: # Todos los links que contengan 'fhv_tripdata' en su texto
    grey_taxis.append(link)
  elif 'fhvhv_tripdata' in link: # Todos los links que contengan 'fhvhv_tripdata' en su texto
    black_taxis.append(link)

yellow_taxis = sorted(yellow_taxis, reverse=True)
green_taxis = sorted(green_taxis, reverse=True)
grey_taxis = sorted(grey_taxis, reverse=True)
black_taxis = sorted(black_taxis, reverse=True)

In [15]:
for link in black_taxis[:2]:
  print(link)
  df = pd.read_parquet(link)
  df = df[['pickup_datetime','PULocationID','DOLocationID']].drop_duplicates().reset_index(drop=True)


https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2019-12.parquet
https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2019-12.parquet


In [ ]:
df = pd.read_parquet(yellow_taxis[0])

# Calcular el uso de memoria en bytes
memory_usage = df.memory_usage(deep=True).sum()

# Estimar el tamaño en disco suponiendo compresión en Parquet (aproximadamente 50% de compresión)
parquet_size = memory_usage * 0.5

print(f"Estimación de tamaño en disco en formato Parquet: {parquet_size} bytes")
print(f"Estimación de tamaño en disco en formato Parquet: {parquet_size / (1024 ** 2):.2f} MB")

Estimación de tamaño en disco en formato Parquet: 750831311.0 bytes
Estimación de tamaño en disco en formato Parquet: 716.05 MB


---
# PySpark

In [ ]:
# Creo una instancia de Spark
spark = SparkSession.builder.appName("ParquetToPandas").getOrCreate()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
spark.conf.set("spark.sql.parquet.enableVectorizedReader", "false")

NameError: ignored

In [ ]:
yellow_taxis[0]

'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2019-12.parquet'

In [ ]:
pd.read_parquet(yellow_taxis[0]).to_parquet('yellow_taxis_0')

In [ ]:
df_spark = spark.read.parquet(yellow_taxis[0], header=True, inferSchema = True)
df_spark.count()

Py4JJavaError: ignored

In [ ]:
spark.stop()


In [ ]:
green_taxis[0]

'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2019-12.parquet'